In [3]:
import os
# import pandas as pd
import numpy as np
import cv2

import joblib
from joblib import dump

import sys
sys.path.append("..")
# import utils.classifier_utils as clf
# from utils.imutils import jimshow as show
# from utils.imutils import jimshow_channel as show_channel

# Machine learning stuff
from sklearn import metrics
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

from tensorflow.keras.datasets import cifar10

# plotting tool
import matplotlib.pyplot as plt

# import argparse

2024-03-15 12:54:58.659877: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-15 12:54:58.663736: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-15 12:54:58.718584: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-15 12:54:59.767693: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [7]:
# This function loads the data from the cifar10 dataset
def load_data():
    return cifar10.load_data()

# This function preprocesses the images
def preprocess_images(images):
    image_list = []  # Empty list which stores the flattened images
    
    for image in images:
        image_grey = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # Converts the images into greyscale
        image_scaled = image_grey / 255.0 # Scales the images
        image_flattened = image_scaled.flatten()  # Flattens the images
        image_list.append(image_flattened)  # Appends the flattened images to the image_list
    images_processed = np.array(image_list) # Converts the list of flattened images to a np array
    return images_processed

# This function trains the logistic regression model
def train_model(X_train_processed, y_train_processed):
        return MLPClassifier(activation = "relu", # maybe change this to ReLU?
                            hidden_layer_sizes = (128,), # make bigger hidden layer
                            max_iter=1000,
                            random_state = 42, early_stopping=True).fit(X_train_processed, y_train_processed)

# This function evaluates the performance of the trained classifier on the test dataset and produces a classification report
def evaluate_model(y_test_processed, X_test_processed, classifier, labels):
    return metrics.classification_report(y_test_processed, classifier.predict(X_test_processed), target_names=labels)
    

# This function saves the classification report and the logistic regression classifier model
def saving_report(classifier_metrics, classifier, report_path, model_path):

    # Opens the file in the out folder in write mode and writes the classification metrics to it.
    with open(report_path, "w") as file:
        file.write(classifier_metrics)
    
    # Saves the trained classifier in the models folder
    joblib.dump(classifier, model_path)

def main():
    # Creates a filepath for each directory 
    out_folder_path = os.path.join("..", "out", "neural_network")
    models_folder_path = os.path.join("..","models", "neural_network")

    # If the directory does not exist, make the directory
    os.makedirs(out_folder_path, exist_ok=True)
    os.makedirs(models_folder_path, exist_ok=True)

    # Filepath for each saved file
    model_path = os.path.join("..", "models", "neural_network", "neural_network_classifier.joblib")
    report_path = os.path.join("..","out", "neural_network", "classification_report.txt")
    # plot_path = os.path.join("..","out", "logistic_regression", "_______")
  
    # Loading the data
    (X_train, y_train), (X_test, y_test) = load_data()

    # List of labels
    labels = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

    # Preprocessing the training and test images
    X_train_processed = preprocess_images(X_train)
    X_test_processed = preprocess_images(X_test)
    y_train_processed = y_train.flatten()
    y_test_processed = y_test.flatten()

    #Training the neural network 
    classifier = train_model(X_train_processed, y_train_processed)

    # Creating the classification report
    classifier_metrics = evaluate_model(y_test_processed, X_test_processed, classifier, labels)

    print(classifier_metrics)
    # # Saving the classification report and the neural network model
    saving_report(classifier_metrics, classifier, report_path, model_path)

if __name__ == "__main__":
    main()

/home/ucloud/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


AttributeError: 'MLPClassifier' object has no attribute '_best_coefs'